## RAG Application - English

## 1. Setup

Importing Libraries

In [3]:
! pip install langchain_openai langchain whisper langchain_community scikit-learn langchain_pinecone langchain[docarray] docarray pydantic==1.10.8 pytube  python-dotenv tiktoken ruff --quiet pandas

Loading the environment variables we need to use.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

Let's define the LLM model that we'll use as part of the workflow.

In [5]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

## 2. Loading the document

In [6]:
with open("kb_en.txt") as file:
    transcription = file.read()

transcription[:100]

'In the ancient kingdom of Hastinapura, a great epic was unfolding, one that would shape the destiny '

## 3. Chunking the document

In [7]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("kb_en.txt")
text_documents = loader.load()
text_documents

[Document(metadata={'source': 'kb_en.txt'}, page_content="In the ancient kingdom of Hastinapura, a great epic was unfolding, one that would shape the destiny of the world and impart timeless lessons. This epic was the Mahabharata, a tale of conflict, duty, righteousness, and destiny.\nThe story began with the Kuru dynasty, ruled by King Shantanu. Shantanu married to Ganga, the river goddess, and they had a son named Devavrata, later known as Bhishma. Bhishma took a vow of celibacy to ensure his father's happiness, allowing Shantanu to marry Satyavati, a fisherwoman. Satyavati bore two sons, Chitrangada and Vichitravirya. Chitrangada died young, and Vichitravirya became king. He married two princesses, Ambika and Ambalika, but died childless. To continue the lineage, Vyasa, Satyavati's son from an earlier union, fathered sons with the queens through Niyoga. Thus, Dhritarashtra, born blind, and Pandu, who had a pale complexion, were born.\nDhritarashtra married Gandhari, who blindfolded 

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)

## 4. Creating embeddings

In [9]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

## 5. Setting up a Vector Store

In [10]:
from langchain_pinecone import PineconeVectorStore

index_name = "tmf2"

pinecone = PineconeVectorStore.from_documents(
    documents, embeddings, index_name=index_name
)

c:\Tools\Python\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


--------------------------------------------------    INGESTION COMPLETES HERE     -----------------------------------------------------------

## 6. Chaining

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# Define the output parser
parser = StrOutputParser()

# Define the prompt template
template = """
Answer the question based on the context below. If you can't answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

retrieved_contexts = []

# Function to log context
def log_context(inputs):
    context_content = [input.page_content for input in inputs['context']]
    retrieved_contexts.append(context_content)
    return inputs

# Chain the prompt, model, and parser
chain = {
    "context": pinecone.as_retriever(search_kwargs={"k": 2}), 
    "question": RunnablePassthrough()
} | RunnablePassthrough(log_context) | prompt | model | parser

## 7. Quering

In [12]:
import pandas as pd
df_1 = pd.read_csv('input_dataset.csv')
df = df_1[20:40:]

# Initialize lists for queries and responses
queries = df['question']
responses = []

In [13]:
df

,question,ground_truth,contexts,answer
20,Who was the ruler of Hastinapura at the beginn...,King Shantanu.,NaN,NaN
21,"What vow did Bhishma take, and why?",Bhishma took a vow of celibacy to ensure his f...,NaN,NaN
22,Who were the sons of King Shantanu and Satyavati?,Chitrangada and Vichitravirya.,NaN,NaN
23,How did Dhritarashtra and Pandu come into exis...,Dhritarashtra and Pandu were born through Niyo...,NaN,NaN
24,What was the significance of Kunti’s divine bo...,Kunti used the divine boons to bear sons from ...,NaN,NaN
25,"Who were the five Pandavas, and who were their...",The five Pandavas were Yudhishthira (son of Dh...,NaN,NaN
26,How did Drona become the teacher of the Kaurav...,Drona was appointed as their teacher to impart...,NaN,NaN
27,What led to the rivalry between Arjuna and Karna?,"Karna, initially an unknown warrior, challenge...",NaN,NaN
28,What was the role of Draupadi in the Pandavas'...,Draupadi became the common wife of all five Pa...,NaN,NaN
29,What happened during the game of dice between ...,"Yudhishthira lost everything to Duryodhana, in...",NaN,NaN


In [14]:
# Loop through each query, invoke response and store in responses list
for query in queries:
    response = chain.invoke(query)  # Replace with your actual chain.invoke(Query) function
    responses.append(response)

In [15]:
# Completing DataFrame
df['answer'] = responses
df['contexts'] = retrieved_contexts

C:\Users\hp\AppData\Local\Temp\ipykernel_18304\3643552263.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['answer'] = responses
C:\Users\hp\AppData\Local\Temp\ipykernel_18304\3643552263.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['contexts'] = retrieved_contexts


In [16]:
pd.set_option('display.max_colwidth', None)
df

,question,ground_truth,contexts,answer
20,Who was the ruler of Hastinapura at the beginning of the Mahabharata?,King Shantanu.,"[In the ancient kingdom of Hastinapura, a great epic was unfolding, one that would shape the destiny, After Pandu's death, Kunti and the Pandavas returned to Hastinapura. Dhritarashtra, now the ruler,]",Dhritarashtra was the ruler of Hastinapura at the beginning of the Mahabharata.
21,"What vow did Bhishma take, and why?","Bhishma took a vow of celibacy to ensure his father's happiness and to allow his father, Shantanu, to marry Satyavati.","[as Bhishma. Bhishma took a vow of celibacy to ensure his father's happiness, allowing Shantanu to, imparted wisdom to Yudhishthira, guiding him through his grief and guilt. Bhishma, lying on his bed]","Bhishma took a vow of celibacy to ensure his father's happiness, allowing Shantanu to."
22,Who were the sons of King Shantanu and Satyavati?,Chitrangada and Vichitravirya.,"[Shantanu to marry Satyavati, a fisherwoman. Satyavati bore two sons, Chitrangada and Vichitravirya., heroic Abhimanyu, Arjuna's son, and Ghatotkacha, Bhima's son.]",Chitrangada and Vichitravirya.
23,How did Dhritarashtra and Pandu come into existence?,"Dhritarashtra and Pandu were born through Niyoga, a practice where Vyasa fathered children with the queens Ambika and Ambalika due to Vichitravirya's death.","[After Pandu's death, Kunti and the Pandavas returned to Hastinapura. Dhritarashtra, now the ruler,, and Pandavas grew intense, especially between Duryodhana and Bhima.]",Dhritarashtra and Pandu came into existence through birth in the context provided.
24,What was the significance of Kunti’s divine boons in the Mahabharata?,"Kunti used the divine boons to bear sons from gods, thus ensuring the birth of the Pandavas: Yudhishthira, Bhima, and Arjuna.","[Kunti invoked divine boons to bear sons. She bore Yudhishthira from Dharma (god of righteousness),, origins were mysterious, but he was eventually revealed as Kunti's firstborn, abandoned at birth]","Kunti's divine boons allowed her to bear sons from different gods, including Yudhishthira from Dharma."
25,"Who were the five Pandavas, and who were their mothers?","The five Pandavas were Yudhishthira (son of Dharma), Bhima (son of Vayu), Arjuna (son of Indra), Nakula, and Sahadeva (sons of the Ashwini twins). Kunti was the mother of Yudhishthira, Bhima, and Arjuna, while Madri was the mother of Nakula and Sahadeva.","[gods). Madri bore Nakula and Sahadeva from the Ashwini twins. These five were the Pandavas., the common wife of all five Pandavas, honoring their bond.]","The five Pandavas were Yudhishthira, Bhima, Arjuna, Nakula, and Sahadeva. Their mothers were Kunti (mother of Yudhishthira, Bhima, and Arjuna) and Madri (mother of Nakula and Sahadeva)."
26,How did Drona become the teacher of the Kaurava and Pandava princes?,Drona was appointed as their teacher to impart advanced military skills.,"[As the princes came of age, Drona, a master of advanced military arts, was appointed their teacher., their teacher. Arjuna, the third Pandava, became Drona's favorite due to his unmatched skills.]",Drona became the teacher of the Kaurava and Pandava princes because he was a master of advanced military arts.
27,What led to the rivalry between Arjuna and Karna?,"Karna, initially an unknown warrior, challenged Arjuna at Draupadi’s swayamvara. Karna's resentment stemmed from his lowly upbringing despite his royal blood, and Duryodhana's support of Karna deepened the rivalry.","[unmatched skills. Duryodhana's jealousy of Arjuna's prowess festered, sowing seeds of discord., a battle of epic proportions between the Pandavas and Kauravas.]",Duryodhana's jealousy of Arjuna's prowess and unmatched skills led to the rivalry between Arjuna and Karna.
28,What was the role of Draupadi in the Pandavas' lives?,Draupadi became the common wife of all five Pandavas due to a misunderstanding at her swayamvara. Her humiliation in the Kaurava court was a key event leading 

-------------------------------------------------------RAG SYSTEM COMPLETES HERE-----------------------------------------------------------

## Evaluation

In [17]:
! pip install ragas nest_asyncio --quiet

In [18]:
import os
from ragas import evaluate
import pandas as pd
from datasets import Dataset
from ragas.metrics import faithfulness, answer_correctness
import nest_asyncio

nest_asyncio.apply()

In [19]:
# Create a Dataset object from the data samples 
evaluation_dataset = Dataset.from_pandas(df)
evaluation_dataset

Dataset({
    features: ['question', 'ground_truth', 'contexts', 'answer'],
    num_rows: 20
})

In [20]:
# Evaluate the dataset using the specified metrics
score = evaluate(evaluation_dataset, metrics=[faithfulness, answer_correctness])

# Convert the score to pandas DataFrame for better visualization
results = score.to_pandas()

Evaluating: 100%|██████████| 40/40 [00:22<00:00,  1.75it/s]


In [ ]:
results.to_csv('results_en.csv', index=False)
results

,question,ground_truth,contexts,answer,faithfulness,answer_correctness
0,Who was the ruler of Hastinapura at the beginning of the Mahabharata?,King Shantanu.,"[In the ancient kingdom of Hastinapura, a great epic was unfolding, one that would shape the destiny, After Pandu's death, Kunti and the Pandavas returned to Hastinapura. Dhritarashtra, now the ruler,]","At the beginning of the Mahabharata, the ruler of Hastinapura was Dhritarashtra.",0.000000,0.210141
1,"What vow did Bhishma take, and why?","Bhishma took a vow of celibacy to ensure his father's happiness and to allow his father, Shantanu, to marry Satyavati.","[as Bhishma. Bhishma took a vow of celibacy to ensure his father's happiness, allowing Shantanu to, imparted wisdom to Yudhishthira, guiding him through his grief and guilt. Bhishma, lying on his bed]","Bhishma took a vow of celibacy to ensure his father's happiness, allowing Shantanu to.",0.666667,0.495538
2,Who were the sons of King Shantanu and Satyavati?,Chitrangada and Vichitravirya.,"[Shantanu to marry Satyavati, a fisherwoman. Satyavati bore two sons, Chitrangada and Vichitravirya., heroic Abhimanyu, Arjuna's son, and Ghatotkacha, Bhima's son.]",The sons of King Shantanu and Satyavati were Chitrangada and Vichitravirya.,0.000000,0.225402
3,How did Dhritarashtra and Pandu come into existence?,"Dhritarashtra and Pandu were born through Niyoga, a practice where Vyasa fathered children with the queens Ambika and Ambalika due to Vichitravirya's death.","[After Pandu's death, Kunti and the Pandavas returned to Hastinapura. Dhritarashtra, now the ruler,, and Pandavas grew intense, especially between Duryodhana and Bhima.]",Pandu and Dhritarashtra were born from Vichitravirya and his wives through the practice of niyoga.,0.000000,0.538979
4,What was the significance of Kunti’s divine boons in the Mahabharata?,"Kunti used the divine boons to bear sons from gods, thus ensuring the birth of the Pandavas: Yudhishthira, Bhima, and Arjuna.","[Kunti invoked divine boons to bear sons. She bore Yudhishthira from Dharma (god of righteousness),, origins were mysterious, but he was eventually revealed as Kunti's firstborn, abandoned at birth]","Kunti's divine boons allowed her to bear sons from different gods, which played a significant role in the lineage and story of the Mahabharata.",0.000000,0.535006
5,"Who were the five Pandavas, and who were their mothers?","The five Pandavas were Yudhishthira (son of Dharma), Bhima (son of Vayu), Arjuna (son of Indra), Nakula, and Sahadeva (sons of the Ashwini twins). Kunti was the mother of Yudhishthira, Bhima, and Arjuna, while Madri was the mother of Nakula and Sahadeva.","[gods). Madri bore Nakula and Sahadeva from the Ashwini twins. These five were the Pandavas., the common wife of all five Pandavas, honoring their bond.]","The five Pandavas were Yudhishthira, Bhima, Arjuna, Nakula, and Sahadeva. Their mothers were Kunti, Madri, and the Ashwini twins.",0.625000,0.769316
6,How did Drona become the teacher of the Kaurava and Pandava princes?,Drona was appointed as their teacher to impart advanced military skills.,"[As the princes came of age, Drona, a master of advanced military arts, was appointed their teacher., their teacher. Arjuna, the third Pandava, became Drona's favorite due to his unmatched skills.]",Drona became the teacher of the Kaurava and Pandava princes because he was a master of advanced military arts.,1.000000,0.612446
7,What led to the rivalry between Arjuna and Karna?,"Karna, initially an unknown warrior, challenged Arjuna at Draupadi’s swayamvara. Karna's resentment stemmed from his lowly upbringing despite his royal blood, and Duryodhana's support of Karna deepened the rivalry.","[unmatched skills. Duryodhana's jealousy of Arjuna's prowess festered, sowing seeds of discord., a battle of epic proportions between the Pandavas and Kauravas.]",Duryodhana's jealousy of Arjuna's prowess and skills led to the rivalry between Arjuna and Karna.,0.000000,0.231763
8,What wa